In [ ]:
import pandas as pd
import time
import networkx as nx
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [2]:
dfaudit = pd.read_pickle('dfaudit3')
dfwork = pd.read_pickle('dfwork3')

In [3]:
g = nx.read_gpickle('graphSimple2.gpickle')

In [4]:
# DataFrame de empresas
dfwork[u'Fecha constitución'] = dfwork[u'Fecha constitución'].str.encode('utf-8')
dfwork.Nombre = dfwork.Nombre.str.encode('utf-8')
dfwork.Estado = dfwork.Estado.str.encode('utf-8')
dfwork.Estado = dfwork.Estado.fillna('Desconocido')
dfwork[u'Incidencias Judiciales'] = dfwork[u'Incidencias Judiciales'].str.encode('utf-8')
dfwork[u'Incidencias Judiciales'] = dfwork[u'Incidencias Judiciales'].fillna('0')
dfwork[u'Reclamaciones administrativas'] = dfwork[u'Reclamaciones administrativas'].str.encode('utf-8')
dfwork[u'Reclamaciones administrativas'] = dfwork[u'Reclamaciones administrativas'].fillna('0')
dfwork[u'Código primario CNAE 2009'] = dfwork[u'Código primario CNAE 2009'].str.encode('utf-8')
dfwork[u'Fecha constitución'] = dfwork[u'Fecha constitución'].fillna('0')

# DataFrame de auditores
dfaudit = dfaudit.dropna(subset=[u'Nombre auditor',u'Salvedades'], how='any')

In [5]:
def crear_nodo_auditor_x(dfaudit,g):
    try:
        g.add_node(dfaudit[u'Nombre auditor'], nodetype='auditor')
    except Exception as e:
        print 'ERROR:',e.args,':','create_node_auditor_x:en la creacion de nodo AUDITOR:',dfaudit[u'Nombre auditor']

def crear_relacion_auditor_x(dfaudit,g):
    try:
        g.add_edge(dfaudit[u'Número BvD'], dfaudit[u'Nombre auditor'], edgetype='AUDITOR', salvedades=dfaudit.Salvedades)
    except Exception as e:
        print 'ERROR:',e.args,':','crear_relacion_auditor_x:en la creacion de la relacion',auditor[u'Nombre auditor']

In [ ]:
def add_empresa_attributes(dfwork,g):
    if dfwork[u'Número BvD'] in g.nodes():
        g.node[dfwork[u'Número BvD']]['cif'] = dfwork[u'Código NIF']
        g.node[dfwork[u'Número BvD']]['fecha_const'] = dfwork[u'Fecha constitución']
        g.node[dfwork[u'Número BvD']]['estado'] = dfwork['Estado']
        g.node[dfwork[u'Número BvD']]['cnae'] = dfwork[u'Código primario CNAE 2009']
        g.node[dfwork[u'Número BvD']]['inc_judic'] = dfwork[u'Incidencias Judiciales']
        g.node[dfwork[u'Número BvD']]['recl_admin'] = dfwork[u'Reclamaciones administrativas']
        g.node[dfwork[u'Número BvD']]['long_x'] = dfwork[u'Coordenada - X']
        g.node[dfwork[u'Número BvD']]['lat_y'] = dfwork[u'Coordenada - Y']

In [ ]:
time1 = time.time()
# Pegamos los atributos
dfwork.apply(add_empresa_attributes, axis=1, args=(g,))

time2 = time.time()

nx.write_gpickle(g, 'FraudAnalysisGraph.gpickle')

print 'Function took %0.3f ms' % ((time2-time1)*1000.0)

In [ ]:
# Creamos los nodos auditor y sus relaciones
dfaudit.apply(crear_nodo_auditor_x, axis=1, args=(g,))
dfaudit.apply(crear_relacion_auditor_x, axis=1, args=(g,))
